In [1]:
import os
import sys

import numpy as np
import pandas as pd

In [2]:
sys.path.append(os.path.abspath('../src'))

from read_and_write_docs import read_jsonl, write_jsonl, read_rds
from utils import apply_temp_doc_id, build_metadata_df
from lambdaG import lambdaG, lambdaG_perplexity
from performance import performance

In [3]:
def compute_perplexity(logprobs):
    """
    Compute sentence-level perplexity from token log-probabilities.
    Assumes log-probs are natural logs (base e), as provided by Qwen‑2.5.
    """
    return np.exp(-np.mean(logprobs))

In [5]:
base_loc = "/Volumes/BCross/datasets/author_verification"

data_type = "test"
corpus = "Enron"

model = "Qwen2.5-1.5B-Instruct"
model_name = model.lower().replace("-", "_")
token_type = model

known_loc = f"{base_loc}/sentence_log_probs_datasets/{data_type}/{corpus}/known_sentence_logprobs_{model_name}.jsonl"
known = read_jsonl(known_loc)
known.rename(columns={'sentence': 'text'}, inplace=True)
known = apply_temp_doc_id(known)
known = known[known['num_tokens'] > 0]
known['perplexity'] = known['log_probs'].apply(compute_perplexity)

unknown_loc = f"{base_loc}/sentence_log_probs_datasets/{data_type}/{corpus}/unknown_sentence_logprobs_{model_name}.jsonl"
unknown = read_jsonl(unknown_loc)
unknown.rename(columns={'sentence': 'text'}, inplace=True)
unknown = apply_temp_doc_id(unknown)
unknown = unknown[unknown['num_tokens'] > 0]
unknown['perplexity'] = unknown['log_probs'].apply(compute_perplexity)

metadata_loc = f"{base_loc}/{data_type}/metadata.rds"
metadata = read_rds(metadata_loc)
filtered_metadata = metadata[metadata['corpus'] == corpus]
agg_metadata = build_metadata_df(filtered_metadata, known, unknown)

In [6]:
# known_filtered = known[known['author'].isin(['Hodja_Nasreddin', 'ZjarriRrethues'])]

In [7]:
# results = lambdaG_perplexity(unknown, known, known, agg_metadata)
# results_agg = (
#     results
#     .groupby(['problem', 'target'], as_index=False)
#     ['score']
#     .mean()
# )
# score_col = 'score'
# target_col = 'target'
# performance(results_agg,
#             score_col,
#             target_col,
#             additional_metadata={
#                 'corpus': corpus
#             })

In [ ]:
all_results = []

for rep in range(1, 6): 
    print(f"Repetition {rep}")
    df = lambdaG(unknown, known, known, agg_metadata)
    # Add the repetition column at the start:
    df.insert(0, 'repetition', rep)
    df.insert(1, 'corpus', corpus)      # move corpus next
    df.insert(2, 'data_type', data_type)
    df.insert(2, 'token_type', token_type) 
    all_results.append(df)

# Combine all repetitions into one DataFrame
results = pd.concat(all_results, ignore_index=True)

Repetition 1
    There are 48 known author(s) and 96 problem(s) in the dataset.
        Working on problem 1 of 96: Kevin.hyatt vs Kevin.hyatt


In [ ]:
save_loc = f"{base_loc}/lambda_g_results/{corpus}_{data_type}_{model_name}_raw.jsonl"
write_jsonl(results, save_loc)

In [ ]:
training_loc = f"{base_loc}/lambda_g_results/{corpus}_training_{model_name}_raw.jsonl"
training = read_jsonl(training_loc)

test_loc = f"{base_loc}/lambda_g_results/{corpus}_test_{model_name}_raw.jsonl"
test = read_jsonl(test_loc)

In [ ]:
training_results_agg = (
    training
    .groupby(['problem', 'target'], as_index=False)
    ['score']
    .mean()
)

test_results_agg = (
    test
    .groupby(['problem', 'target'], as_index=False)
    ['score']
    .mean()
)

In [ ]:
score_col = 'score'
target_col = 'target'

In [ ]:
result_metrics = performance(training_results_agg,
                             score_col,
                             target_col,
                             df_test=test_results_agg,
                             additional_metadata={
                                 'corpus': corpus
                             })

In [ ]:
result_metrics